## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

### Load Dataset Files

Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

In [0]:
#project_path = "/content/drive/My Drive/AIML_Project/Dog_breed/"
project_path="/content/drive/My Drive/Colab Notebooks/DogBreed_Classification/"

Run the below code to extract all the images in the train.zip files given in the dataset. We are going to use these images as train and validation sets and their labels in further steps.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'train.zip', 'r') as z:
  z.extractall()

Repeat the same step for test.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'test.zip', 'r') as z:
  z.extractall()

Repeat the same step for sample_submission.csv.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'sample_submission.csv.zip', 'r') as z:
  z.extractall()

Repeat the same step for labels.csv.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'labels.csv.zip', 'r') as z:
  z.extractall()

After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

### Read labels.csv file using pandas

In [0]:
import pandas as pd
labels=pd.read_csv('labels.csv')

In [10]:
len(labels)
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


### Print the count of each category of Dogs given in the dataset



In [0]:
INPUT_SIZE = 224
NUM_CLASSES = 16
SEED = 1987
sample_submission=pd.read_csv('sample_submission.csv')
selected_breed_list = list(labels.groupby('breed').count().sort_values(by='id', ascending=False).head(NUM_CLASSES).index)

### Get one-hot encodings of labels

In [0]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
enc = OneHotEncoder(handle_unknown='ignore')
data = enc.fit_transform(labels).toarray() 

target=pd.Series(labels['breed'])
one_hot=pd.get_dummies(target,sparse=True)
target=np.asarray(one_hot)

## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [0]:
import cv2
im_size=100
ids=labels['id']
X_train=[]
y_train=target
for id in ids:
  img=cv2.imread('./train/{}'.format(id)+'.jpg')
  X_train.append(cv2.resize(img,(im_size,im_size)))
X_train=np.array(X_train,np.float32)

Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [0]:
def standardize(array):
  array/=255
  return array
X_train=standardize(X_train)


In [14]:
X_train.shape

(10222, 100, 100, 3)

### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [0]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(X_train, y_train, test_size=0.20)

### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [0]:
dataset_test=pd.read_csv('sample_submission.csv')
ids_test=dataset_test['id']

Run the below code to load the test image files in x_test_feature

In [0]:
#x_test_feature = []
#i = 0 # initialisation
#for f in tqdm(test_img.values): # f for format ,jpg
#    img = cv2.imread('./test/{}.jpg'.format(f), 0)
#    img_resize = cv2.resize(img, (img_rows, img_cols)) 
#    x_test_feature.append(img_resize)
im_size=100
X_test=list()
for id in ids_test:
  img=cv2.imread('./test/{}'.format(id)+'.jpg')
  X_test.append(cv2.resize(img,(im_size,im_size)))
X_test=np.array(X_test,np.float32)

In [18]:
X_test.shape

(10357, 100, 100, 3)

Normalize the test data and convert it into 4 dimensions

In [0]:
X_test=standardize(X_test)


In [20]:
X_test.shape

(10357, 100, 100, 3)

### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [22]:


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam

print(X_train.shape,y_train.shape,X_test.shape)



(10222, 100, 100, 3) (10222, 120) (10357, 100, 100, 3)


### Use batch_size = 128 and epochs = 10 and execute the model

In [39]:
batch_size=128
epochs=10
num_classes = 120
model=Sequential()
input_shape=train_x.shape[1:]
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2))) 
model.add(Flatten())
model.add(Dense(256, activation='relu')) 
model.add(Dense(num_classes, activation='softmax'))
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 96, 96, 32)        2432      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 44, 44, 64)        51264     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 22, 22, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 30976)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               7930112   
_________________________________________________________________
dense_5 (Dense)              (None, 120)              

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [41]:
#model.fit(X_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1)
model.fit(train_x,train_y,batch_size=batch_size,epochs=epochs,verbose=1)

Epoch 1/10
64/64 [==============================] - 4s 56ms/step - loss: 4.7920 - accuracy: 0.0127
Epoch 2/10
64/64 [==============================] - 4s 57ms/step - loss: 4.5743 - accuracy: 0.0368
Epoch 3/10
64/64 [==============================] - 4s 56ms/step - loss: 3.9644 - accuracy: 0.1230
Epoch 4/10
64/64 [==============================] - 4s 57ms/step - loss: 3.0190 - accuracy: 0.2923
Epoch 5/10
64/64 [==============================] - 4s 56ms/step - loss: 1.7514 - accuracy: 0.5764
Epoch 6/10
64/64 [==============================] - 4s 56ms/step - loss: 0.7590 - accuracy: 0.8194
Epoch 7/10
64/64 [==============================] - 4s 57ms/step - loss: 0.2837 - accuracy: 0.9391
Epoch 8/10
64/64 [==============================] - 4s 57ms/step - loss: 0.1242 - accuracy: 0.9780
Epoch 9/10
64/64 [==============================] - 4s 56ms/step - loss: 0.0705 - accuracy: 0.9868
Epoch 10/10
64/64 [==============================] - 4s 57ms/step - loss: 0.0814 - accuracy: 0.9848


In [0]:
score = model.evaluate(test_x, test_y, verbose=0)

In [28]:
print('\n', 'Test accuracy:', score[1])


 Test accuracy: 0.0347188264131546


#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip = True,
        channel_shift_range = 0.20)
test_datagen = ImageDataGenerator(rescale=1./255)

### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

In [70]:
train_generator = train_datagen.flow_from_directory(
    directory=r"train/",
    target_size=(100, 100),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=False,
    seed=42
    
)
test_generator = test_datagen.flow_from_directory(
    directory=r"./test",
    target_size=(100, 100),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=False,
    seed=42
)
optimizer = Adam(lr=0.0001)

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

In [0]:
batch_size=128
epochs=10
num_classes = 120
model=Sequential()
input_shape=train_x.shape[1:]
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2))) 
model.add(Flatten())
model.add(Dense(256, activation='relu')) 
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit_generator(
        train_generator,
        steps_per_epoch=len(X_train) ,
        epochs=10,
        validation_data=test_generator,
       )
score = model.evaluate(test_x, test_y, verbose=0)


# Model accuracy is still poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [29]:
from keras.applications.vgg16 import VGG16, preprocess_input
# Instantiate the model with the pre-trained weights (no top)
base_model= VGG16(weights=(project_path+'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'),
                 include_top=False, pooling='avg')

Using TensorFlow backend.


Print the summary of the base_model

In [30]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

In [0]:
input_shape=X_train.shape[1:]
model2=Sequential()
for layer in model.layers:
  model2.add(layer)
model2.add(Flatten())
model2.add(Dense(1024, input_shape=input_shape, activation='relu', name='fc1'))
model2.add(Dense(256, activation='relu', name='fc2'))
model2.add(Dense(120, activation='softmax', name='output'))

### Make all the layers in the base_model (VGG16) to be non-trainable

In [0]:
for layer in model2.layers:
    if layer.name != 'fc1' or layer.name!='fc2' or layer.name != 'output':
        layer.trainable = False

### Fit and compile the model with batch_size = 128 and epochs = 10 and execute the model

Try to get training and validation accuracy to be more than 90%

In [33]:
model2.compile(optimizer = 'adam', loss= 'categorical_crossentropy', metrics=['accuracy'])
model2.fit(train_x, train_y  ,batch_size=128,epochs=10, verbose = 1)

Epoch 1/10
64/64 [==============================] - 2s 26ms/step - loss: 4.7859 - accuracy: 0.0175
Epoch 2/10
64/64 [==============================] - 2s 26ms/step - loss: 4.7859 - accuracy: 0.0175
Epoch 3/10
64/64 [==============================] - 2s 26ms/step - loss: 4.7859 - accuracy: 0.0175
Epoch 4/10
64/64 [==============================] - 2s 26ms/step - loss: 4.7859 - accuracy: 0.0175
Epoch 5/10
64/64 [==============================] - 2s 26ms/step - loss: 4.7859 - accuracy: 0.0175
Epoch 6/10
64/64 [==============================] - 2s 25ms/step - loss: 4.7859 - accuracy: 0.0175
Epoch 7/10
64/64 [==============================] - 2s 25ms/step - loss: 4.7859 - accuracy: 0.0175
Epoch 8/10
64/64 [==============================] - 2s 26ms/step - loss: 4.7859 - accuracy: 0.0175
Epoch 9/10
64/64 [==============================] - 2s 25ms/step - loss: 4.7859 - accuracy: 0.0175
Epoch 10/10
64/64 [==============================] - 2s 25ms/step - loss: 4.7859 - accuracy: 0.0175


In [0]:
score = model2.evaluate(test_x, test_y, verbose=0)

In [36]:
print('\n', 'Test accuracy:', score[1])


 Test accuracy: 0.01026894897222519


In [43]:
X_test.shape

(10357, 100, 100, 3)

In [44]:
X_train.shape

(10222, 100, 100, 3)